In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os

In [2]:
path_customers = "../h&m dataset/customer_hm.csv"
path_articles = "../h&m dataset/articles_hm.csv"
path_transactions = "../h&m dataset/transactions_hm.csv"

df_customers = pd.read_csv(path_customers)
df_articles = pd.read_csv(path_articles)
df_transactions = pd.read_csv(path_transactions)


- 결측치 

 대부분 정제되어있기때문에 결측치가 없었지만 상세설명란에 수백 수천개의 결측치가있었고, 해당 부분은 데이터셋에 영향을 주지않는 행들로 포함하기로 결정

- df_transactions에 0개
- df_customers 에 1개 (fashion_news(1)) -> 포함 (사용하지않는 컬럼)
- df_articles에 416개 (detail_desc(416)) -> 포함 (있어도 없어도 영향없는 컬럼)

In [5]:
# 결측치 처리(포함) 전 행의 개수 [articles]
count1 = len(df_articles)
print(f'처리 전 ariticles 행의 개수 {count1}')

# 결측치 처리(포함) 전 행의 개수 [customers]
count2 = len(df_customers)
print(f'처리 전 customers 행의 개수 {count2}')

처리 전 ariticles 행의 개수 105542
처리 전 customers 행의 개수 1048575


In [6]:
# articles에 상세설명 Unknown으로 포함
df_articles['detail_desc'] = df_articles['detail_desc'].fillna('Unknown')

# customers에 fashion_news_frequency NONE으로 포함
df_customers['fashion_news_frequency'] = df_customers['fashion_news_frequency'].fillna('NONE')
print('---결측치 포함---')

# 결측치 처리(포함) 후 행의 개수 
count1 = len(df_articles)
print(f'처리 후 articles 행의 개수 {count1}')
count2 = len(df_customers)
print(f'처리 후 customers 행의 개수 {count2}')

---결측치 포함---
처리 후 articles 행의 개수 105542
처리 후 customers 행의 개수 1048575


2. 이상치 범위 설정 

- H&m에서 공식적으로 사용하는 연령대 세그먼트는 (16-24 , 25-34, 35+) 16세 이하 80세 이상은 이상치로 범주 아웃 
- 또한 'age' 컬럼에 결측치는 0으로 포함해 범주 인 하려했으나 결측치가 없는 데이터였음 

- 나이 구간 설정

In [7]:
biz_bins = [0, 15, 24, 34, 90]
biz_labels = ['Non-Target (<16)', '16-24 (Z세대)', '25-34 (밀레니얼)', '35+ (X세대 이상)'] 

df_customers['age_group_biz'] = pd.cut(
    df_customers['age'], 
    bins=biz_bins, 
    labels=biz_labels, 
    right=True,
    include_lowest=True
)

print('최종 비즈니스 세그먼트 (age_group_biz) 생성 완료!')
print("--- 그룹별 고객 수 ---")
print(df_customers['age_group_biz'].value_counts().sort_index())


최종 비즈니스 세그먼트 (age_group_biz) 생성 완료!
--- 그룹별 고객 수 ---
age_group_biz
Non-Target (<16)         0
16-24 (Z세대)         276631
25-34 (밀레니얼)        304493
35+ (X세대 이상)        467399
Name: count, dtype: int64


In [8]:
initial_count = len(df_customers)


mask = (df_customers['age'] >= 16) & (df_customers['age'] <= 90)
df_customers = df_customers[mask].copy()


final_count = len(df_customers)


removed_count = initial_count - final_count


print(f"1. 초기 고객 수 (제거 전): {initial_count:,}명")
print(f"2. 제거된 이상치: {removed_count:,}명")
print(f"3. 이후 고객 수 (제거 후): {final_count:,}명")


1. 초기 고객 수 (제거 전): 1,048,575명
2. 제거된 이상치: 52명
3. 이후 고객 수 (제거 후): 1,048,523명


- 가격대가 0원인 데이터는 이상치로 분별해 삭제시키는 방향으로 

- 가격대가 너무 높은 데이터는 특별히 가격을 목적에 두지않는 이상 포함시키기로 결정

In [9]:
df_trans_working = df_transactions.copy()
initial_count = len(df_trans_working)

#  0원 거래 건수 확인
zero_price_mask = df_trans_working['price'] == 0
zero_count = zero_price_mask.sum()

df_trans_working = df_trans_working[df_trans_working['price'] > 0].copy() 
final_count_after_zero = len(df_trans_working)


# 4. 상위 0.1% 가격 기준점 계산
high_price_threshold = df_trans_working['price'].quantile(0.999) 
high_price_count = (df_trans_working['price'] > high_price_threshold).sum()

print('-----------')
print(f"1. 초기 거래 건수 (제거 전): {initial_count:,}건")
print(f"2. 제거된 거래 건수 (가격 0원): {zero_count:,}건")
print(f"3. 확인된 초고가 거래 (상위 0.1%): {high_price_count:,}건")
print(f"4. 최종 분석 대상 거래 건수 (제거 후): {final_count_after_zero:,}건")

-----------
1. 초기 거래 건수 (제거 전): 1,048,575건
2. 제거된 거래 건수 (가격 0원): 0건
3. 확인된 초고가 거래 (상위 0.1%): 871건
4. 최종 분석 대상 거래 건수 (제거 후): 1,048,575건


- 결측치/이상치 최종 저장

In [10]:
df_customers_cleaned = df_customers.copy()
df_articles_cleaned = df_articles.copy()
df_transactions_cleaned = df_transactions.copy()

In [ ]:
PROCESSED_DIR = os.path.join('..', '전처리완료')

# 저장할 폴더가 없으면 생성 (안정장치용도)
if not os.path.exists(PROCESSED_DIR):
    os.mkdir(PROCESSED_DIR)
    print(f"폴더 '{PROCESSED_DIR}'를 생성했습니다.")

# --- 데이터 저장 (.to_parquet) ---
# df_customers_cleaned, df_transactions_cleaned, df_articles가 메모리에 있어야함

df_customers_cleaned.to_parquet(os.path.join(PROCESSED_DIR, 'customers_clean.parquet'))
df_transactions_cleaned.to_parquet(os.path.join(PROCESSED_DIR, 'transactions_clean.parquet'))
df_articles.to_parquet(os.path.join(PROCESSED_DIR, 'articles_clean.parquet'))

print("---" * 15)
print("---모든 전처리 데이터가 Parquet 형식으로 저장되었는지---.")

---------------------------------------------
모든 전처리 데이터가 Parquet 형식으로 저장 완료되었습니다.
